In [ ]:
using CSV
using DataFrames
using DataFramesMeta

## Parse Data

In [ ]:
# Mappings and default values

fuel_map = Dict(
    "Natural Gas Fired Combined Cycle" => "NATURAL_GAS",
    "Natural Gas Fired Combustion Turbine" => "NATURAL_GAS",
    "Natural Gas Internal Combustion Engine" => "NATURAL_GAS",
    "Petroleum Liquids" => "DISTILLATE_FUEL_OIL",
    "Petroleum Coke" => "PETROLEUM_COKE",
    "Nuclear" => "NUCLEAR",
    "Conventional Steam Coal" => "COAL",
    "Coal Integrated Gasification Combined Cycle" => "COAL",
    "Natural Gas Steam Turbine" => "NATURAL_GAS",
    "Wood/Wood Waste Biomass" => "WOOD_WASTE",
    "Other Waste Biomass" => "OTHER",
    "Other Gases" => "OTHER",
    "Municipal Solid Waste" => "MUNICIPAL_WASTE",
    "Landfill Gas" => "OTHER_GAS",
    "All Other" => "OTHER",
    "Solar Photovoltaic" => "SOLAR",
    "Onshore Wind Turbine" => "WIND",
    "Hydroelectric Pumped Storage" => "HYDRO",
    "Conventional Hydroelectric" => "HYDRO",
    "Batteries" => "CHEMICAL",
)

ap_min_factor = Dict(
    "Natural Gas Fired Combined Cycle" => 0.5,
    "Natural Gas Fired Combustion Turbine" => 0.5,
    "Natural Gas Internal Combustion Engine" => 0.5,
    "Petroleum Liquids" => 0.5,
    "Petroleum Coke" => 0.5,
    "Nuclear" => 0.8,
    "Conventional Steam Coal" => 0.8,
    "Coal Integrated Gasification Combined Cycle" => 0.8,
    "Natural Gas Steam Turbine" => 0.5,
    "Wood/Wood Waste Biomass" => 0.5,
    "Other Waste Biomass" => 0.5,
    "Other Gases" => 0.5,
    "Municipal Solid Waste" => 0.5,
    "Landfill Gas" => 0.5,
    "All Other" => 0.5,
    "Solar Photovoltaic" => 0.0,
    "Onshore Wind Turbine" => 0.0,
    "Hydroelectric Pumped Storage" => 0.0,
    "Conventional Hydroelectric" => 0.0,
    "Batteries" => 0.0
)

fuel_price = Dict(
    "Natural Gas Fired Combined Cycle" => 4.0,
    "Natural Gas Fired Combustion Turbine" => 4.0,
    "Natural Gas Internal Combustion Engine" => 4.0,
    "Petroleum Liquids" => 10.0,
    "Petroleum Coke" => 10,
    "Nuclear" => 1.0,
    "Conventional Steam Coal" => 2,
    "Coal Integrated Gasification Combined Cycle" => 2,
    "Natural Gas Steam Turbine" => 4.0,
    "Wood/Wood Waste Biomass" => 10.0,
    "Other Waste Biomass" => 10.0,
    "Other Gases" => 10.0,
    "Municipal Solid Waste" => 10.0,
    "Landfill Gas" => 4.0,
    "All Other" => 20.0,
    "Solar Photovoltaic" => 0.0,
    "Onshore Wind Turbine" => 0.0,
    "Hydroelectric Pumped Storage" => 0.0,
    "Conventional Hydroelectric" => 0.0,
    "Batteries" => 0.0
)

fors = CSV.read("../../data-open/miso_gen/miso_fors.csv", DataFrame)

function getFOR(technology::String, cap::Float64, lookupTable::DataFrame)
    res = @chain lookupTable begin
        @rsubset :technology == technology
        @rsubset :min_cap <= cap && cap < :max_cap
        @select :average
    end

    return (res[1, 1])
end

eia_metadata = CSV.read("../../data-open/miso_gen/eia_plant_data.csv", DataFrame)
@select! eia_metadata :eia_plant_code :eia_generator_id :ap_min

epa_heat_rates = CSV.read("../../data-open/miso_gen/epa_heat_rates.csv", DataFrame)
@select! epa_heat_rates :eia_plant_code :plant_nominal_heat_rate_btu_kwh

# getFOR("Natural Gas Internal Combustion Engine", 45, fors)

misc_attrs = DataFrame([
    (fuel = "WOOD_WASTE", prime_mover = "ST", min_up_time = 2, min_down_time = 4, ramp_limits = 1, default_hr = 0.0),
    (fuel = "WIND", prime_mover = "WT", min_up_time = 0, min_down_time = 0, ramp_limits = -1, default_hr = 0.0),
    (fuel = "SOLAR", prime_mover = "PV", min_up_time = 0, min_down_time = 0, ramp_limits = -1, default_hr = 0.0),
    (fuel = "PETROLEUM_COKE", prime_mover = "ST", min_up_time = 2, min_down_time = 4, ramp_limits = 1, default_hr = 14410.0),
    (fuel = "OTHER_GAS", prime_mover = "CA", min_up_time = 2, min_down_time = 4, ramp_limits = 3, default_hr = 10000.0),
    (fuel = "OTHER_GAS", prime_mover = "CT", min_up_time = 1, min_down_time = 1, ramp_limits = 1, default_hr = 10000.0),
    (fuel = "OTHER_GAS", prime_mover = "IC", min_up_time = 1, min_down_time = 1, ramp_limits = 1, default_hr = 10000.0),
    (fuel = "OTHER_GAS", prime_mover = "GT", min_up_time = 2, min_down_time = 4, ramp_limits = 3, default_hr = 10000.0),
    (fuel = "OTHER", prime_mover = "ST", min_up_time = 0, min_down_time = 0, ramp_limits = -1, default_hr = 14410.0),
    (fuel = "OTHER", prime_mover = "OT", min_up_time = 0, min_down_time = 0, ramp_limits = -1, default_hr = 14410.0),
    (fuel = "OTHER", prime_mover = "IC", min_up_time = 0, min_down_time = 0, ramp_limits = -1, default_hr = 14410.0),
    (fuel = "NUCLEAR", prime_mover = "ST", min_up_time = 24, min_down_time = 48, ramp_limits = 20, default_hr = 10000.0),
    (fuel = "NATURAL_GAS", prime_mover = "CC", min_up_time = 8, min_down_time = 5, ramp_limits = 4, default_hr = 10000.0),
    (fuel = "NATURAL_GAS", prime_mover = "CT", min_up_time = 2, min_down_time = 2, ramp_limits = 4, default_hr = 10000.0),
    (fuel = "NATURAL_GAS", prime_mover = "CA", min_up_time = 8, min_down_time = 5, ramp_limits = 4, default_hr = 10000.0),
    (fuel = "NATURAL_GAS", prime_mover = "CS", min_up_time = 8, min_down_time = 5, ramp_limits = 4, default_hr = 10000.0),
    (fuel = "NATURAL_GAS", prime_mover = "GT", min_up_time = 2, min_down_time = 2, ramp_limits = 4, default_hr = 10000.0),
    (fuel = "NATURAL_GAS", prime_mover = "IC", min_up_time = 2, min_down_time = 2, ramp_limits = 4, default_hr = 10000.0),
    (fuel = "NATURAL_GAS", prime_mover = "ST", min_up_time = 8, min_down_time = 5, ramp_limits = 4, default_hr = 10000.0),
    (fuel = "MUNICIPAL_WASTE", prime_mover = "ST", min_up_time = 2, min_down_time = 4, ramp_limits = 1, default_hr = 10000.0),
    (fuel = "HYDRO", prime_mover = "HY", min_up_time = 0, min_down_time = 0, ramp_limits = -1, default_hr = 10000.0),
    (fuel = "HYDRO", prime_mover = "PS", min_up_time = 0, min_down_time = 0, ramp_limits = -1, default_hr = 10000.0),
    (fuel = "DISTILLATE_FUEL_OIL", prime_mover = "GT", min_up_time = 2, min_down_time = 4, ramp_limits = 3, default_hr = 14410.0),
    (fuel = "DISTILLATE_FUEL_OIL", prime_mover = "IC", min_up_time = 1, min_down_time = 1, ramp_limits = 3, default_hr = 14410.0),
    (fuel = "COAL", prime_mover = "ST", min_up_time = 48, min_down_time = 24, ramp_limits = 4, default_hr = 10000.0),
    (fuel = "COAL", prime_mover = "CT", min_up_time = 48, min_down_time = 24, ramp_limits = 4, default_hr = 10000.0),
    (fuel = "COAL", prime_mover = "CA", min_up_time = 48, min_down_time = 24, ramp_limits = 4, default_hr = 10000.0),
    (fuel = "CHEMICAL", prime_mover = "BA", min_up_time = 0, min_down_time = 0, ramp_limits = -1, default_hr = 0.0),
])
;

In [ ]:
function parse_generation()
    generators = CSV.read("../../data-open/miso_gen/miso_generators_raw.csv", DataFrame);

    @rsubset! generators :Existing_or_Planned == "Existing"
    @rsubset! generators :Operational_Year <= 2024
    @rsubset! generators :MISO_Region == "South" || :MISO_Region == "EastCentral" || :MISO_Region == "West"
    @rename! generators begin
        :technology = :Technology
        :prime_mover = :Prime_Mover
        :capacity = :Capacity_MW
        :region = :MISO_Region
        :eia_plant_code = :EIA_Plant_ID
        :eia_generator_id = :EIA_Generator_ID
    end
    @rtransform! generators :name = lowercase(replace(strip(:Plant_Name), " " => "_")) * "_" * string(:eia_plant_code) * "_" * :eia_generator_id
    @select! generators :name :technology :prime_mover :capacity :region :eia_plant_code :eia_generator_id

    dispatchable_gen = @chain generators begin
        @rsubset :technology != "Solar Photovoltaic" && :technology != "Onshore Wind Turbine" && :technology != "Conventional Hydroelectric" && :technology != "All Other"
    end

    pv_gen = @chain generators begin
        @rsubset :technology == "Solar Photovoltaic"
        @groupby :region
        @combine :region_capacity = sum(:capacity)
        @rtransform :name = "solar_" * lowercase(:region)
        @rtransform :technology = "Solar Photovoltaic"
        @rtransform :prime_mover = "PV"
        @rtransform :eia_plant_code = -9999
        @rtransform :eia_generator_id = -9999
    end

    pv_gen = @chain pv_gen begin
        @rename :capacity = :region_capacity
        @select :name :technology :prime_mover :capacity :region :eia_plant_code :eia_generator_id
    end

    wind_gen = @chain generators begin
        @rsubset :technology == "Onshore Wind Turbine"
        @groupby :region
        @combine :region_capacity = sum(:capacity)
        @rtransform :name = "wind_" * lowercase(:region)
        @rtransform :technology = "Onshore Wind Turbine"
        @rtransform :prime_mover = "WT"
        @rtransform :eia_plant_code = -9999
        @rtransform :eia_generator_id = -9999
    end

    wind_gen = @chain wind_gen begin
        @rename :capacity = :region_capacity
        @select :name :technology :prime_mover :capacity :region :eia_plant_code :eia_generator_id
    end

    wind_gen = @chain generators begin
        @rsubset :technology == "Onshore Wind Turbine"
        @groupby :region
        @combine :region_capacity = sum(:capacity)
        @rtransform :name = "wind_" * lowercase(:region)
        @rtransform :technology = "Onshore Wind Turbine"
        @rtransform :prime_mover = "WT"
        @rtransform :eia_plant_code = -9999
        @rtransform :eia_generator_id = -9999
    end

    wind_gen = @chain wind_gen begin
        @rename :capacity = :region_capacity
        @select :name :technology :prime_mover :capacity :region :eia_plant_code :eia_generator_id
    end

    hydro_gen = @chain generators begin
        @rsubset :technology == "Conventional Hydroelectric"
        @groupby :region
        @combine :region_capacity = sum(:capacity)
        @rtransform :name = "hydro_" * lowercase(:region)
        @rtransform :technology = "Conventional Hydroelectric"
        @rtransform :prime_mover = "HY"
        @rtransform :eia_plant_code = nothing
        @rtransform :eia_generator_id = nothing
    end

    hydro_gen = @chain hydro_gen begin
        @rename :capacity = :region_capacity
        @select :name :technology :prime_mover :capacity :region :eia_plant_code :eia_generator_id
    end

    all_gen = reduce(
        vcat,
        [
            dispatchable_gen,
            pv_gen,
            wind_gen,
            hydro_gen
        ]
    )

    # mean_time_to_recovery = 10 #hours
    @rtransform! all_gen :for = getFOR(:technology, :capacity, fors) / 100
    # @rtransform! all_gen :recovery_transition_probability = 1 / mean_time_to_recovery
    # @rtransform! all_gen :outage_transition_probability = (:for * :recovery_transition_probability) / (1 - :for)

    @rtransform! all_gen :bus = lowercase(:region)

    @rtransform! all_gen :fuel = fuel_map[:technology]

    all_gen = leftjoin(all_gen, misc_attrs, on=[:fuel, :prime_mover])
    @rtransform! all_gen :ramp_limits = :ramp_limits >= 0 ? :ramp_limits : :capacity

    all_gen = leftjoin(all_gen, eia_metadata, on=[:eia_plant_code, :eia_generator_id])
    @rtransform! all_gen :ap_min = ismissing(:ap_min) ? get(ap_min_factor, :technology, 0.5) * :capacity : :ap_min

    all_gen = leftjoin(all_gen, epa_heat_rates, on=[:eia_plant_code])
    @rtransform! all_gen :heat_rate_incr_1 = ismissing(:plant_nominal_heat_rate_btu_kwh) ? :default_hr : abs(:plant_nominal_heat_rate_btu_kwh)

    ### Defaults

    @rtransform! all_gen :active_power = 0.0
    @rtransform! all_gen :reactive_power = 0.0
    @rtransform! all_gen :ap_max = :capacity
    @rtransform! all_gen :base_mva = 1

    @rtransform! all_gen :fuel_price = fuel_price[:technology]
    @rtransform! all_gen :output_point_0 = 1.0
    

    # @rtransform! all_gen :min_down_time = 0.0
    # @rtransform! all_gen :min_up_time = 0.0

    @select! all_gen :name :bus :prime_mover :fuel :active_power :reactive_power :capacity :ap_min :ap_max :base_mva :for :fuel_price :output_point_0 :heat_rate_incr_1 :min_down_time :min_up_time :ramp_limits

    # all_gen[findall(nonunique(all_gen, :name)), :]
    all_gen = unique(all_gen, :name)

    CSV.write("./data/gen.csv", all_gen)
    return(all_gen)    
end

In [ ]:
parse_generation()

In [ ]:
function parse_timeseries(ts::DataFrame, year::String) 
    # ts = CSV.read("../../data-open/miso_gen/MISO Fuel Mix by Region $year.csv", DataFrame);

    @select! ts :interval_start_est :region :fuel_type :rt_generation_state_estimator
    @rename! ts :DateTime = :interval_start_est 

    ts.rt_generation_state_estimator = coalesce.(ts.rt_generation_state_estimator, 0.0)

    solar = @rsubset ts :fuel_type == "Solar"
    solar = unstack(solar, :region, :rt_generation_state_estimator)
    @rename! solar begin
        :solar_west = :North
        :solar_eastcentral = :Central
        :solar_south = :South
    end
    @select! solar Not(:fuel_type)
    CSV.write("./ts-data/gen/solar-$year.csv", solar)

    wind = @rsubset ts :fuel_type == "Wind"
    wind = unstack(wind, :region, :rt_generation_state_estimator)
    @rename! wind begin
        :wind_west = :North
        :wind_eastcentral = :Central
        :wind_south = :South
    end
    @select! wind Not(:fuel_type)
    CSV.write("./ts-data/gen/wind-$year.csv", wind)

    hydro = @rsubset ts :fuel_type == "Hydro"
    hydro = unstack(hydro, :region, :rt_generation_state_estimator)
    @rename! hydro begin
        :hydro_west = :North
        :hydro_eastcentral = :Central
        :hydro_south = :South
    end
    @select! hydro Not(:fuel_type)
    CSV.write("./ts-data/gen/hydro-$year.csv", hydro)

    load = @chain ts begin 
        @groupby :DateTime :region
        @combine :load = sum(:rt_generation_state_estimator)
    end
    load = unstack(load, :region, :load)
    @rename! load begin
        :west = :North
        :eastcentral = :Central
        :south = :South
    end
    CSV.write("./ts-data/load/load-$year.csv", load)
end

In [ ]:
# year = "2022"
# ts = CSV.read("../../data-open/miso_gen/miso_fuel_mix_by_region_$year.csv", DataFrame);
# parse_timeseries(ts, year)